# The OpenMMTools Monte Carlo Engine

While it is very common to use molecular dynamics to generate the paths used in path sampling, a path can be any ordered sequence of snapshots. For example, one could generate a sequence of Markov Chain Monte Carlo steps connecting two states, and do path sampling on that.

For those interested in doing this (within the domain of force-field based simulation), the OpenMMTools project has a subpackage for Markov Chain Monte Carlo, and OpenPathSampling has an engine that uses that subpackage.

As usual, you set up the engine just as you would for a normal simulation with it. Then we wrap things from the underlying tool in the OPS engine wrapper.

## Creating an MCMC sampler with OpenMMTools

In [ ]:
from simtk import unit
import openmmtools
from openmmtools import testsystems, cache, mcmc
from openmmtools.states import ThermodynamicState, SamplerState

In [ ]:
testsystem = testsystems.AlanineDipeptideVacuum()
thermodynamic_state = ThermodynamicState(system=testsystem.system, 
                                         temperature=298*unit.kelvin)

In the OpenMMTools MCMC package, each move applies to all the atoms in its `atom_subset`. So to create a move that randomly selects a single atom and randomly displaces that atom, you need to create a displacement move for each atom, then join them in a `WeightedMove`.

In [ ]:
moves = [
    mcmc.MCDisplacementMove(displacement_sigma=0.1*unit.angstrom,
                            atom_subset=[i])
    for i in range(testsystem.mdtraj_topology.n_atoms)
]
move = mcmc.WeightedMove([(m, 1/len(moves)) for m in moves])

In [ ]:
# use OpenMMTools to get good initial conditions
sampler_state = SamplerState(positions=testsystem.positions)
sampler = mcmc.MCMCSampler(thermodynamic_state, sampler_state, move)
sampler.minimize()

## Setting up path sampling with OpenPathSampling

In [ ]:
import openpathsampling as paths
from openpathsampling.engines.openmm.mcengine import OpenMMToolsMCEngine, snapshot_from_sampler_state

# we'll use the new storage, because it is faster
from openpathsampling.experimental.storage import monkey_patch_all, Storage
from openpathsampling.experimental.storage.collective_variables import MDTrajFunctionCV
paths = monkey_patch_all(paths)

import mdtraj as md
import numpy as np

### Creating the OPS engine

The next two cells are the only ones specific to integrating this new engine type with OPS.

In [ ]:
mdtraj_topology = testsystem.mdtraj_topology
ops_topology = paths.engines.MDTrajTopology(mdtraj_topology)

In [ ]:
engine = OpenMMToolsMCEngine(thermodynamic_state, move,
                             {'n_steps_per_frame': 100,
                              'n_frames_max': 1000},
                             topology=ops_topology).named("engine")

### Defining CVs and stable states

In [ ]:
# just to verify that we have the right atoms
[mdtraj_topology.atom(i) for i in [4, 6, 8, 14, 16]]

In [ ]:
# CVs
phi = MDTrajFunctionCV(md.compute_dihedrals, topology=ops_topology, 
                       period_min=-np.pi, period_max=np.pi,
                       indices=[[4, 6, 8, 14]]).named("phi")
psi = MDTrajFunctionCV(md.compute_dihedrals, topology=ops_topology, 
                       period_min=-np.pi, period_max=np.pi,
                       indices=[[6, 8, 14, 16]]).named("psi")

In [ ]:
# TODO: check these values for Amber ff96
# estimates based on eyeballing https://doi.org/10.1098/rspa.2019.0036
# Mediocre state defs won't mess up the sampling, but aren't as helpful in teaching
C7eq = (
    paths.PeriodicCVDefinedVolume(phi, lambda_min=-np.pi, lambda_max=-0.8,
                                  period_min=-np.pi, period_max=np.pi)
    & paths.PeriodicCVDefinedVolume(psi, lambda_min=0.5, lambda_max=3.5,
                                    period_min=-np.pi, period_max=np.pi)
).named("C7eq")
C7ax = (
    paths.PeriodicCVDefinedVolume(phi, lambda_min=0.5, lambda_max=1.5,
                                  period_min=-np.pi, period_max=np.pi)
    & paths.PeriodicCVDefinedVolume(psi, lambda_min=-2.0, lambda_max=-0.5,
                                    period_min=-np.pi, period_max=np.pi)
).named("C7ax")
# period information in CVDefinedVolumes won't be required in OPS 2.0

### Creating sampling network and move scheme

For TPS, these are very easy. They get more complicated for TIS.

In [ ]:
network = paths.TPSNetwork(C7eq, C7ax).named("tps-network")
scheme = paths.OneWayShootingMoveScheme(network, engine=engine).named('one-way TPS')

### Getting an initial transition trajectory

This is always one of the hardest parts of setting up a TPS simulation. As it happens, alanine dipeptide is a relatively ballistic system (velocity memory can matter) so it can be easier to generate a high temperature trajectory with molecular dynamics than with Monte Carlo. Therefore, we'll use MD to create our initial trajectory.

In [ ]:
from openmmtools import integrators
hi_temp = paths.engines.openmm.Engine(
    topology=ops_topology,
    system=testsystem.system,
    integrator=integrators.VVVRIntegrator(temperature=750*unit.kelvin,
                                          collision_rate=1.0 / unit.picosecond,
                                          timestep=2.0 * unit.femtosecond),
    options={'n_steps_per_frame': 10,
             'n_frames_max': 20000}
)

In [ ]:
# make a snapshot from the minimized sampler state
snapshot = snapshot_from_sampler_state(sampler.sampler_state)

In [ ]:
visit_all = paths.VisitAllStatesEnsemble([C7ax, C7eq])
try:
    traj = hi_temp.generate(snapshot, visit_all.can_append)
except paths.engines.EngineError as e:
    traj = e.last_trajectory  # for debugging
    raise

In [ ]:
# extract only the transition using scheme.initial_conditions_from_trajectories
init_conds = scheme.initial_conditions_from_trajectories(traj)

### Equilibrating our initial trajectory

In [ ]:
equil = paths.PathSampling(
    storage=Storage("mc_equil.db", mode='w'),
    move_scheme=scheme,
    sample_set=init_conds
)
equil.run_until_decorrelated()

In [ ]:
decorrelated = equil.sample_set

### Putting together the path sampling simulation

In [ ]:
storage = Storage("mc_tps.db", mode='w')

In [ ]:
tps = paths.PathSamplingling(
    storage=storage,
    move_scheme=scheme,
    sample_set=init_conds
)

In [ ]:
tps.run(100)